In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from gPhoton.pipeline import execute_pipeline

Pipeline steps:
1. ~Run photometry with extraction on NUV~
2. ~Run photometry with extraction on FUV~
3. ~Run photometry on FUV with NUV source positions~
4. ~Run photometry on NUV with FUV source positions~
5. ~Generate a merged catalog from NUV extractions~
6. ~Generate a merged catalog from FUV extractions~
7. Generate a merged catalog from all NUV & FUV detections

In [2]:
from glcat_merge_utils import *
from lightcurve_utils import *
from glcat_catalog_utils import *

In [3]:
data = accumulate_run_data(23456)
cat_nuv = make_catalog(data,'NUV')
cat_fuv = make_catalog(data,'FUV')

In [4]:
def crossmatch_catalogs(master_table, match_table,
                        match_radius=2.5, # arcseconds
                       ):
    master_catalog = SkyCoord(ra=master_table['RA'].values*u.degree,
                              dec=master_table['DEC'].values*u.degree)
    match_catalog = SkyCoord(ra=match_table['RA'].values*u.degree,
                             dec=match_table['DEC'].values*u.degree)
    catalog_ix, d2d, d3d = match_catalog.match_to_catalog_sky(master_catalog)
    sep_constraint = d2d < match_radius * u.arcsec

    return catalog_ix,d2d

In [6]:
# MATCH FUV TO NUV

idx,d2d=crossmatch_catalogs(cat_nuv,cat_fuv)
match = pd.DataFrame({'idx':idx,'d2d':d2d,
                      'good_match':np.array(d2d <= 2.5 * u.arcsec,dtype=int),
                      'best_match':np.zeros(len(idx),dtype=int)})
for ix in np.unique(idx):
    best_ix = np.argmin(match.loc[match['idx']==ix]['d2d'].values)
    match.loc[match.loc[match['idx']==ix].iloc[best_ix].name,'best_match']=1

# add indexes to the FUV catalog that point at the NUV matches

cat_fuv['F2N_NUV_MATCH_INDEX']=np.full(len(cat_fuv),-1)
cat_fuv['F2N_MATCH_DISTANCE']=np.full(len(cat_fuv),np.nan)
cat_fuv['F2N_GOOD_MATCH']=np.full(len(cat_fuv),-1)
cat_fuv['F2N_BEST_MATCH']=np.full(len(cat_fuv),-1)

cat_fuv.loc[match.index,'F2N_NUV_MATCH_INDEX']=np.array(match['idx'].values,dtype=int)
cat_fuv.loc[match.index,'F2N_MATCH_DISTANCE']=match['d2d'].values
cat_fuv.loc[match.index,'F2N_GOOD_MATCH']=np.array(match['good_match'].values,dtype=int)
cat_fuv.loc[match.index,'F2N_BEST_MATCH']=np.array(match['best_match'].values,dtype=int)

# add indexes to the NUV catalog that point at the FUV matches

cat_nuv['F2N_FUV_MATCH_INDEX']=np.full(len(cat_nuv),-1)
cat_nuv['F2N_MATCH_DISTANCE']=np.full(len(cat_nuv),np.nan)
cat_nuv['F2N_GOOD_MATCH']=np.full(len(cat_nuv),-1)
cat_nuv['F2N_BEST_MATCH']=np.full(len(cat_nuv),-1)

cat_nuv.loc[match['idx'],'F2N_FUV_MATCH_INDEX']=np.array(np.arange(len(match)),dtype=int)
cat_nuv.loc[match['idx'],'F2N_MATCH_DISTANCE']=match['d2d'].values
cat_nuv.loc[match['idx'],'F2N_GOOD_MATCH']=np.array(match['good_match'].values,dtype=int)
cat_nuv.loc[match['idx'],'F2N_BEST_MATCH']=np.array(match['best_match'].values,dtype=int)


In [13]:
idx,d2d=crossmatch_catalogs(cat_fuv,cat_nuv)
idx,d2d,len(idx)

(array([   1,    1,    1, ..., 3454, 3454, 3454]),
 <Angle [0.01403541, 0.01042539, 0.01162781, ..., 0.01041329, 0.00924208,
         0.00959142] deg>,
 4840)

In [14]:
# MATCH NUV TO FUV

idx,d2d=crossmatch_catalogs(cat_fuv,cat_nuv)
match = pd.DataFrame({'idx':idx,'d2d':d2d,
                      'good_match':np.array(d2d <= 2.5 * u.arcsec,dtype=int),
                      'best_match':np.zeros(len(idx),dtype=int)})
for ix in np.unique(idx):
    best_ix = np.argmin(match.loc[match['idx']==ix]['d2d'].values)
    match.loc[match.loc[match['idx']==ix].iloc[best_ix].name,'best_match']=1

# add indexes to the NUV catalog that point at the FUV matches

cat_fuv['F2N_NUV_MATCH_INDEX']=np.full(len(cat_fuv),-1)
cat_fuv['F2N_MATCH_DISTANCE']=np.full(len(cat_fuv),np.nan)
cat_fuv['F2N_GOOD_MATCH']=np.full(len(cat_fuv),-1)
cat_fuv['F2N_BEST_MATCH']=np.full(len(cat_fuv),-1)

cat_fuv.loc[match.index,'F2N_NUV_MATCH_INDEX']=np.array(match['idx'].values,dtype=int)
cat_fuv.loc[match.index,'F2N_MATCH_DISTANCE']=match['d2d'].values
cat_fuv.loc[match.index,'F2N_GOOD_MATCH']=np.array(match['good_match'].values,dtype=int)
cat_fuv.loc[match.index,'F2N_BEST_MATCH']=np.array(match['best_match'].values,dtype=int)

# add indexes to the NUV catalog that point at the FUV matches

cat_nuv['F2N_FUV_MATCH_INDEX']=np.full(len(cat_nuv),-1)
cat_nuv['F2N_MATCH_DISTANCE']=np.full(len(cat_nuv),np.nan)
cat_nuv['F2N_GOOD_MATCH']=np.full(len(cat_nuv),-1)
cat_nuv['F2N_BEST_MATCH']=np.full(len(cat_nuv),-1)

cat_nuv.loc[match['idx'],'F2N_FUV_MATCH_INDEX']=np.array(np.arange(len(match)),dtype=int)
cat_nuv.loc[match['idx'],'F2N_MATCH_DISTANCE']=match['d2d'].values
cat_nuv.loc[match['idx'],'F2N_GOOD_MATCH']=np.array(match['good_match'].values,dtype=int)
cat_nuv.loc[match['idx'],'F2N_BEST_MATCH']=np.array(match['best_match'].values,dtype=int)


KeyError: '[3962, 3963, 3964, 3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973, 3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982, 3983, 3984, 3985, 3986, 3987, 3988, 3989, 3990, 3991, 3992, 3993, 3994, 3995, 3996, 3997, 3998, 3999, 4000, 4001, 4002, 4003, 4004, 4005, 4006, 4007, 4008, 4009, 4010, 4011, 4012, 4013, 4014, 4015, 4016, 4017, 4018, 4019, 4020, 4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031, 4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040, 4041, 4042, 4043, 4044, 4045, 4046, 4047, 4048, 4049, 4050, 4051, 4052, 4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083, 4084, 4085, 4086, 4087, 4088, 4089, 4090, 4091, 4092, 4093, 4094, 4095, 4096, 4097, 4098, 4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107, 4108, 4109, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126

In [15]:
idx

array([   1,    1,    1, ..., 3454, 3454, 3454])